In [1]:
from notebook_inits import *
import polars as pl

In [2]:
from main import fetch_all_data

In [ ]:
data = fetch_all_data()

In [4]:
cb_bids = pl.DataFrame(data['cb']['bids'], schema=['price', 'qty', 'orders'], orient="row")
cb_asks = pl.DataFrame(data['cb']['asks'], schema=['price', 'qty', 'orders'], orient="row")
_, _ = cb_bids.drop_in_place('orders'), cb_asks.drop_in_place('orders')
cb_bids, cb_asks = cb_bids.cast(pl.Float64), cb_asks.cast(pl.Float64)

In [5]:
cb_bids.head(3), cb_asks.head(3)

(shape: (3, 2)
 ┌───────────┬──────────┐
 │ price     ┆ qty      │
 │ ---       ┆ ---      │
 │ f64       ┆ f64      │
 ╞═══════════╪══════════╡
 │ 107033.64 ┆ 0.037372 │
 │ 107033.26 ┆ 0.019826 │
 │ 107030.0  ┆ 0.0039   │
 └───────────┴──────────┘,
 shape: (3, 2)
 ┌───────────┬──────────┐
 │ price     ┆ qty      │
 │ ---       ┆ ---      │
 │ f64       ┆ f64      │
 ╞═══════════╪══════════╡
 │ 107033.65 ┆ 0.021876 │
 │ 107034.0  ┆ 0.0039   │
 │ 107034.23 ┆ 0.019825 │
 └───────────┴──────────┘)

In [6]:
gem_bids, gem_asks = pl.DataFrame(data['gem']['bids']), pl.DataFrame(data['gem']['asks'])
_, _ = gem_bids.drop_in_place('timestamp'), gem_asks.drop_in_place('timestamp')
gem_bids, gem_asks = gem_bids.cast(pl.Float64), gem_asks.cast(pl.Float64)
gem_bids, gem_asks = gem_bids.rename({"amount": "qty"}), gem_asks.rename({"amount": "qty"})

In [7]:
gem_bids.head(3), gem_asks.head(3)

(shape: (3, 2)
 ┌───────────┬──────────┐
 │ price     ┆ qty      │
 │ ---       ┆ ---      │
 │ f64       ┆ f64      │
 ╞═══════════╪══════════╡
 │ 107013.49 ┆ 0.000995 │
 │ 107008.23 ┆ 0.000934 │
 │ 107002.68 ┆ 0.016839 │
 └───────────┴──────────┘,
 shape: (3, 2)
 ┌───────────┬──────────┐
 │ price     ┆ qty      │
 │ ---       ┆ ---      │
 │ f64       ┆ f64      │
 ╞═══════════╪══════════╡
 │ 107013.77 ┆ 0.00467  │
 │ 107013.8  ┆ 0.00467  │
 │ 107019.09 ┆ 0.000975 │
 └───────────┴──────────┘)

### Merge

In [8]:
all_bids = pl.concat((cb_bids, gem_bids)).sort('price', descending=True)
all_asks = pl.concat((cb_asks, gem_asks)).sort('price', descending=False)
all_bids.head(3), all_asks.head(3)

(shape: (3, 2)
 ┌───────────┬──────────┐
 │ price     ┆ qty      │
 │ ---       ┆ ---      │
 │ f64       ┆ f64      │
 ╞═══════════╪══════════╡
 │ 107033.64 ┆ 0.037372 │
 │ 107033.26 ┆ 0.019826 │
 │ 107030.0  ┆ 0.0039   │
 └───────────┴──────────┘,
 shape: (3, 2)
 ┌───────────┬──────────┐
 │ price     ┆ qty      │
 │ ---       ┆ ---      │
 │ f64       ┆ f64      │
 ╞═══════════╪══════════╡
 │ 107013.77 ┆ 0.00467  │
 │ 107013.8  ┆ 0.00467  │
 │ 107019.09 ┆ 0.000975 │
 └───────────┴──────────┘)

In [9]:
all_bids = all_bids.with_columns((pl.col('price') * pl.col('qty')).alias('exec_price'))
all_bids = all_bids.with_columns(pl.col('qty').cum_sum().alias('qty_cumsum'),   pl.col("exec_price").cum_sum().alias("cumsum"))
all_asks = all_asks.with_columns((pl.col('price') * pl.col('qty')).alias('exec_price'))
all_asks = all_asks.with_columns(pl.col('qty').cum_sum().alias('qty_cumsum'),  pl.col("exec_price").cum_sum().alias("cumsum"))
all_bids.head(3), all_asks.head(3)

(shape: (3, 5)
 ┌───────────┬──────────┬────────────┬────────────┬────────────┐
 │ price     ┆ qty      ┆ exec_price ┆ qty_cumsum ┆ cumsum     │
 │ ---       ┆ ---      ┆ ---        ┆ ---        ┆ ---        │
 │ f64       ┆ f64      ┆ f64        ┆ f64        ┆ f64        │
 ╞═══════════╪══════════╪════════════╪════════════╪════════════╡
 │ 107033.64 ┆ 0.037372 ┆ 4000.01731 ┆ 0.037372   ┆ 4000.01731 │
 │ 107033.26 ┆ 0.019826 ┆ 2121.99967 ┆ 0.057197   ┆ 6122.01698 │
 │ 107030.0  ┆ 0.0039   ┆ 417.417    ┆ 0.061097   ┆ 6539.43398 │
 └───────────┴──────────┴────────────┴────────────┴────────────┘,
 shape: (3, 5)
 ┌───────────┬──────────┬────────────┬────────────┬─────────────┐
 │ price     ┆ qty      ┆ exec_price ┆ qty_cumsum ┆ cumsum      │
 │ ---       ┆ ---      ┆ ---        ┆ ---        ┆ ---         │
 │ f64       ┆ f64      ┆ f64        ┆ f64        ┆ f64         │
 ╞═══════════╪══════════╪════════════╪════════════╪═════════════╡
 │ 107013.77 ┆ 0.00467  ┆ 499.754306 ┆ 0.00467    ┆ 49

In [14]:
target_qty = 1
index_pos = all_bids.select(pl.col('qty_cumsum').search_sorted(target_qty)).item()
index_pos, all_bids.slice(index_pos-1, 3)

(23,
 shape: (3, 5)
 ┌───────────┬──────────┬─────────────┬────────────┬───────────────┐
 │ price     ┆ qty      ┆ exec_price  ┆ qty_cumsum ┆ cumsum        │
 │ ---       ┆ ---      ┆ ---         ┆ ---        ┆ ---           │
 │ f64       ┆ f64      ┆ f64         ┆ f64        ┆ f64           │
 ╞═══════════╪══════════╪═════════════╪════════════╪═══════════════╡
 │ 107013.49 ┆ 0.000995 ┆ 106.468791  ┆ 0.996812   ┆ 106680.638874 │
 │ 107012.65 ┆ 0.010744 ┆ 1149.743912 ┆ 1.007556   ┆ 107830.382786 │
 │ 107012.07 ┆ 0.0519   ┆ 5553.880418 ┆ 1.059456   ┆ 113384.263204 │
 └───────────┴──────────┴─────────────┴────────────┴───────────────┘)

In [15]:
index_pos = all_asks.select(pl.col('qty_cumsum').search_sorted(target_qty)).item()
index_pos, all_asks.slice(index_pos-1, 3)

(19,
 shape: (3, 5)
 ┌───────────┬──────────┬──────────────┬────────────┬───────────────┐
 │ price     ┆ qty      ┆ exec_price   ┆ qty_cumsum ┆ cumsum        │
 │ ---       ┆ ---      ┆ ---          ┆ ---        ┆ ---           │
 │ f64       ┆ f64      ┆ f64          ┆ f64        ┆ f64           │
 ╞═══════════╪══════════╪══════════════╪════════════╪═══════════════╡
 │ 107037.73 ┆ 0.342185 ┆ 36626.661755 ┆ 0.901359   ┆ 96475.964347  │
 │ 107037.74 ┆ 0.280275 ┆ 29999.999367 ┆ 1.181634   ┆ 126475.963714 │
 │ 107038.0  ┆ 0.0039   ┆ 417.4482     ┆ 1.185534   ┆ 126893.411914 │
 └───────────┴──────────┴──────────────┴────────────┴───────────────┘)